In [50]:
import SymbolList as sl
import pandas as pd
import datetime as dt 
import fix_yahoo_finance as yf

## 1. Symbol List 초기화 또는 불러오기

In [41]:
df = sl.init_symbol_DF()
sl.add_symbol(df,init=True)

DUP : Apple Inc.
DUP : Apple Inc.
DUP : Microsoft Corporation
DUP : International Business Machines Corporation
DUP : Oracle Corporation
DUP : Amazon.com, Inc.
DUP : Tesla, Inc.
DUP : Samsung Electronics Co., Ltd
DUP : SK hynix, Inc.
DUP : LG Electronics Inc
DUP : NAVER Corporation


,Symbol,From,To
Name,,,
Apple Inc.,AAPL,2010-01-01,2019-05-19
Microsoft Corporation,MSFT,2010-01-01,2019-05-19
International Business Machines Corporation,IBM,2010-01-01,2019-05-19
Oracle Corporation,ORCL,2010-01-01,2019-05-19
"Amazon.com, Inc.",AMZN,2010-01-01,2019-05-19
"Tesla, Inc.",TSLA,2010-01-01,2019-05-19
"Samsung Electronics Co., Ltd",005930.KS,2010-01-01,2019-05-19
"SK hynix, Inc.",000660.KS,2010-01-01,2019-05-19
LG Electronics Inc,066570.KS,2010-01-01,2019-05-19


## 2. Symbol List 에서 데이터 받기

In [43]:
list_df = [None] * len(df.index)

In [46]:
for idx in range(0,len(df.index)):
    list_df[idx] = df.index[idx]
    # print('Note::Processing' + list_df[idx])
    today = dt.datetime.utcnow().strftime("%Y-%m-%d")
    # 이미 최신 데이터
    if(df.iloc[idx][2] == today):
        # print('  Pass::Alread up-to-date.')
        continue
    try:
        temp_df = yf.download(df.iloc[idx][0], start = df.iloc[idx][1], end =  today)
        df.iloc[idx][2] = today
    except ValueError:
        print('Value Error')
    temp_df.to_csv(list_df[idx] + '.csv')
# 심볼리스트 갱신
df.to_csv('symbol_list.csv')
print('Note::symbol_list.csv is updated.')

Note::symbol_list.csv is updated.


## 3. 통계

+ how to calculate coskew and cokurtosis
https://stackoverflow.com/questions/41890870/how-to-calculate-coskew-and-cokurtosis

생각을 좀 해봐야겠다.

$$ K(X,X,Y,Y)= {E[(X - E[X])^2(Y - E[Y]^2)] \over \sigma_X^2\sigma_Y^2}$$

In [61]:
t_df = pd.read_csv(list_df[0] + '.csv',index_col = 'Date')

# 연간 종가
t_data = t_df.iloc[-240:]['Close']
t_data.kurt()

-0.8499127293799393